## Setup - Run this first

In [ ]:
from sqlmodel import Session, select
from app.database import engine
from app.models import User
from app.utils import hash_password
from datetime import datetime
import pandas as pd

print("✅ Imports successful!")

## 1. View All Users

In [ ]:
def get_all_users():
    """Get all users from the database"""
    with Session(engine) as session:
        users = session.exec(select(User)).all()
        
        # Convert to pandas DataFrame for better display
        users_data = []
        for user in users:
            users_data.append({
                'ID': user.id,
                'Username': user.username,
                'Name': user.name,
                'Email': user.email,
                'Phone': user.phone_number,
                'Active': user.is_active,
                'Created': user.created_at.strftime('%Y-%m-%d %H:%M'),
                'Updated': user.updated_at.strftime('%Y-%m-%d %H:%M')
            })
        
        df = pd.DataFrame(users_data)
        return df

# Display all users
users_df = get_all_users()
print(f"Total Users: {len(users_df)}\n")
users_df

## 2. Create New User

In [ ]:
def create_user(username, password, email, name=None, phone_number=None, is_active=True):
    """Create a new user"""
    with Session(engine) as session:
        # Check if username already exists
        existing_user = session.exec(select(User).where(User.username == username)).first()
        if existing_user:
            print(f"❌ Error: Username '{username}' already exists!")
            return None
        
        # Check if email already exists
        existing_email = session.exec(select(User).where(User.email == email)).first()
        if existing_email:
            print(f"❌ Error: Email '{email}' already exists!")
            return None
        
        # Create new user
        new_user = User(
            username=username,
            password_hash=hash_password(password),
            email=email,
            name=name,
            phone_number=phone_number,
            is_active=is_active
        )
        
        session.add(new_user)
        session.commit()
        session.refresh(new_user)
        
        print(f"✅ User created successfully!")
        print(f"   ID: {new_user.id}")
        print(f"   Username: {new_user.username}")
        print(f"   Email: {new_user.email}")
        return new_user

In [ ]:
# Example: Create a new user
# Modify these values as needed
create_user(
    username="john_doe",
    password="SecurePass123",
    email="john@example.com",
    name="John Doe",
    phone_number="+1234567890"
)

## 3. Update Existing User

In [ ]:
def update_user(username, **updates):
    """
    Update an existing user
    
    Available fields to update:
    - name
    - email
    - phone_number
    - is_active
    - password (will be hashed automatically)
    """
    with Session(engine) as session:
        user = session.exec(select(User).where(User.username == username)).first()
        
        if not user:
            print(f"❌ Error: User '{username}' not found!")
            return None
        
        # Update fields
        updated_fields = []
        
        if 'name' in updates:
            user.name = updates['name']
            updated_fields.append(f"name -> {updates['name']}")
        
        if 'email' in updates:
            user.email = updates['email']
            updated_fields.append(f"email -> {updates['email']}")
        
        if 'phone_number' in updates:
            user.phone_number = updates['phone_number']
            updated_fields.append(f"phone_number -> {updates['phone_number']}")
        
        if 'is_active' in updates:
            user.is_active = updates['is_active']
            updated_fields.append(f"is_active -> {updates['is_active']}")
        
        if 'password' in updates:
            user.password_hash = hash_password(updates['password'])
            updated_fields.append("password -> ******")
        
        user.updated_at = datetime.utcnow()
        
        session.add(user)
        session.commit()
        session.refresh(user)
        
        print(f"✅ User '{username}' updated successfully!")
        for field in updated_fields:
            print(f"   {field}")
        
        return user


In [ ]:

# Example: Update a user
# Modify these values as needed
update_user(
    username="john_doe",
    name="John Updated Doe",
    phone_number="+9876543210"
)

In [ ]:
# Run this cell anytime to see updated user list
users_df = get_all_users()
print(f"Total Users: {len(users_df)}\n")
users_df